In [142]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as soup
import time, pickle

import datetime, requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
current_date = datetime.datetime.now()

#### URLs

In [47]:
url_listagem_empresas = "http://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm"
url_search_empresa = "http://bvmf.bmfbovespa.com.br/pt-br/mercados/acoes/empresas/ExecutaAcaoConsultaInfoEmp.asp?CodCVM={CVM_CODE}&ViewDoc=0#a"

### Get available tickers

In [173]:
import json
import requests
from bs4 import BeautifulSoup

In [178]:
class B3EmpresasScraper(object):
    def __init__(self):
        self.search_request = {"State":{},
                               "TabState":{"ctl00_contentPlaceHolderConteudo_tabMenuEmpresaListada_tabNome":{"Selected":True}}}
    def scrape(self):
        jobs = self.scrape_jobs()
        for job in jobs:
            print(job)
                
    def scrape_jobs(self, max_pages=3):
        jobs = []
#         pageno = 0
#         self.search_request['pageNumber'] = pageno

#         while pageno < max_pages:
        payload = { 
            '__EVENTTARGET': 'ctl00:contentPlaceHolderConteudo:BuscaNomeEmpresa1:btnTodas',
            '__VIEWSTATE': '/wEPDwULLTE2OTg3NzU5NjgPZBYCZg9kFgQCAQ9kFgICEA8WAh4EVGV4dAUIbWVyY2Fkb3NkAgMPZBYCAgEPZBYEAgEPZBYCAgEPZBYGAgUPDxYCHwAFCjIxLzExLzIwMTlkZAIJDw8WAh8ABQUyM2gwMGRkAhEPDxYCHgVDb3VudAIEZBYCAgMPDxYGHgJJRAUacHZ3QnVzY2FBbm9MaXN0YWdlbUVtcHJlc2EeB1Rvb2xUaXBlHgdWaXNpYmxlaGQWAgIBD2QWAgIBDxBkZBYAZAIDDw9kFgIeB29uY2xpY2sFHGhpc3RvcnkuYmFjaygpOyByZXR1cm4gZmFsc2VkGAYFPGN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEJ1c2NhU2V0b3JFbXByZXNhMSRncmRTZXRvcg8UKwACZBcAZAVBY3RsMDAkY29udGVudFBsYWNlSG9sZGVyQ29udGV1ZG8kQnVzY2FTZWdtZW50b0VtcHJlc2ExJGdyZEVtcHJlc2EPFCsAAmQXAGQFHl9fQ29udHJvbHNSZXF1aXJlUG9zdEJhY2tLZXlfXxYIBT1jdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYU5vbWVFbXByZXNhMSRncmRFbXByZXNhBTxjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYVNldG9yRW1wcmVzYTEkZ3JkU2V0b3IFPmN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEJ1c2NhU2V0b3JFbXByZXNhMSRncmRFbXByZXNhBUFjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYVNlZ21lbnRvRW1wcmVzYTEkZ3JkRW1wcmVzYQVEY3RsMDAkY29udGVudFBsYWNlSG9sZGVyQ29udGV1ZG8kQnVzY2FBbm9MaXN0YWdlbUVtcHJlc2ExJGdyZEVtcHJlc2EFL2N0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEFqYXhQYW5lbEJ1c2NhBStjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRtcGdQYWdpbmFzBTZjdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyR0YWJNZW51RW1wcmVzYUxpc3RhZGEFRGN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlckNvbnRldWRvJEJ1c2NhQW5vTGlzdGFnZW1FbXByZXNhMSRncmRFbXByZXNhDxQrAAJkFwBkBT5jdGwwMCRjb250ZW50UGxhY2VIb2xkZXJDb250ZXVkbyRCdXNjYVNldG9yRW1wcmVzYTEkZ3JkRW1wcmVzYQ8UKwACZBcAZAU9Y3RsMDAkY29udGVudFBsYWNlSG9sZGVyQ29udGV1ZG8kQnVzY2FOb21lRW1wcmVzYTEkZ3JkRW1wcmVzYQ8UKwACZBcAZILXCPCm5ktP5zeUM9V5d8CeqVJU',
            '__VIEWSTATEGENERATOR': '9CFBB4E5',
            '__EVENTVALIDATION': '/wEWKQKQw/XtAQKrqYeWCwL85ODVCQLh+8LADwKyt5yADgKXzv7qAwLoidiqAgLNoLqVCALuyLaBBALT35jsCQKkm/KrCAKJstSWDgLa7a3WDAK/hJDBAgKQwOmAAQL11svrBgL2pYL/BwLbvOTpDQKs+L2pDAKRj6CUAgLiyvlTAsfh274GApidtf4EAv2zl+kKAp7ck9UGAoPz9b8MAtSuz/8KApb/x9cCAvuVqsIIAszRg4IHArHo5ewMAoKkv6wLAue6oZcBArj2+tYPAp2N3cEFAr612a0BAqPMu5gHAqqynsAFAvrzvPwMAr/S5NkCArSz5PIOQBNPwVxP46TJ4TGUiUDReAcNROw=',
            'ctl00_contentPlaceHolderConteudo_AjaxPanelBuscaPostDataValue': 'ctl00_contentPlaceHolderConteudo_AjaxPanelBusca,ActiveElement,ctl00_contentPlaceHolderConteudo_BuscaNomeEmpresa1_btnTodas;',
            'ctl00$contentPlaceHolderConteudo$tabMenuEmpresaListada': json.dumps(self.search_request),
            'ctl00$contentPlaceHolderConteudo$mpgPaginas_Selected' : '0',
            'RadAJAXControlID': 'ctl00_contentPlaceHolderConteudo_AjaxPanelBusca',
            'httprequest': 'true'
        }

        r = requests.post(
            url='http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/BuscaEmpresaListada.aspx?idioma=pt-br',
            data=payload,
            headers={
                'X-Requested-With': 'HttpRequest'
            }
        )

#         s = BeautifulSoup(r.text)
#             if not s.requisition:
#                 break

#             for r in s.findAll('ctl00_contentPlaceHolderConteudo_tabMenuEmpresaListada_tabNome'):
#                 job = {}
#                 job['jobid'] = r.jobid.text
#                 job['title'] = r.postingtitle and \
#                     r.postingtitle.text or r.retailpostingtitle.text
#                 job['location'] = r.location.text
#                 jobs.append(job)

#             # Next page
#             pageno += 1
#             self.search_request['pageNumber'] = pageno

        return r

In [207]:
scraper = B3EmpresasScraper()
pg = scraper.scrape_jobs()
raw_soup = soup(pg.text, "html.parser")
raw_companies = raw_soup.find('tbody').findAll('tr')

In [249]:
empresas_listadas = {cvm_code:{'name':n,'segment':s} for (cvm_code,n,s) in zip ([int(c.findAll('a')[0]['href'].replace('ResumoEmpresaPrincipal.aspx?codigoCvm=','')) for c in raw_companies],
                                                                  [c.findAll('a')[0].text.strip() for c in raw_companies],
                                                                  [c.findAll('td')[2].text.strip() for c in raw_companies])}

422